# Wczytanie gotowego zestawu przetworzonych danych

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/preprocessed_data.csv')

# Wczytanie predykcji PSE

In [ ]:
df_pse = pd.read_csv('data/predictions_pse.csv')

# Podzial danych na zbiór treningowy, walidacjny i testowy

In [ ]:
from sklearn.model_selection import train_test_split

data = df.copy()

test_24h_data = data.iloc[-48:-24]
data = data.iloc[:-48]

train_data, temp_data = train_test_split(data, test_size=0.1, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)

# Normalizacja danych

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np

scalers = {}
train_scaled = train_data.copy()
for column in train_data.columns:
    scaler = MinMaxScaler()
    train_scaled[column] = scaler.fit_transform(train_data[column].values.reshape(-1, 1))
    scalers[column] = scaler
    joblib.dump(scaler, f'scalers/scaler_{column}.pkl')

def scale_data(data, scalers):
    data_scaled = data.copy()
    for column in data.columns:
        scaler = scalers[column]
        data_scaled[column] = scaler.transform(data[column].values.reshape(-1, 1))
    return data_scaled


val_scaled = scale_data(val_data, scalers)
test_scaled = scale_data(test_data, scalers)
test_24h_scaled = scale_data(test_24h_data, scalers)
test_24h_scaled = np.array([test_24h_scaled])

# Utworzenie sekwencji danych

In [ ]:
def create_sequences(dataset, look_back, forecast_horizon):
    data_values = dataset.values
    num_elements = data_values.shape[0]
    X = [data_values[i:i+look_back] for i in range(num_elements - look_back - forecast_horizon + 1)]
    Y = [data_values[i+look_back:i+look_back+forecast_horizon, 0] for i in range(num_elements - look_back - forecast_horizon + 1)]
    return np.array(X), np.array(Y)


look_back = 24
forecast_horizon = 24

X_train_seq, y_train_seq = create_sequences(train_scaled, look_back, forecast_horizon)
X_val_seq, y_val_seq = create_sequences(val_scaled, look_back, forecast_horizon)
X_test_seq, y_test_seq = create_sequences(test_scaled, look_back, forecast_horizon)

# Definicja funkcji błędów

In [ ]:
import tensorflow as tf
from sklearn.metrics import mean_squared_error

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Metryki błedów oraz wizualizacje predykcji dla poszczególnych modeli

In [ ]:
def plots_and_metrics(model):
    train_predict = model.predict(X_train_seq)
    val_predict = model.predict(X_val_seq)
    test_predict = model.predict(X_test_seq)

    train_predict_original = scalers['electricity_load'].inverse_transform(train_predict)
    y_train_original = scalers['electricity_load'].inverse_transform(y_train_seq)
    val_predict_original = scalers['electricity_load'].inverse_transform(val_predict)
    y_val_original = scalers['electricity_load'].inverse_transform(y_val_seq)
    test_predict_original = scalers['electricity_load'].inverse_transform(test_predict)
    y_test_original = scalers['electricity_load'].inverse_transform(y_test_seq)

    print('Train RMSE:', np.sqrt(mean_squared_error(y_train_original, train_predict_original)))
    print('Validation RMSE:', np.sqrt(mean_squared_error(y_val_original, val_predict_original)))
    print('Test RMSE:', np.sqrt(mean_squared_error(y_test_original, test_predict_original)))
    print('Train MAPE:', mape(y_train_original, train_predict_original))
    print('Validation MAPE:', mape(y_val_original, val_predict_original))
    print('Test MAPE:', mape(y_test_original, test_predict_original))

    predictions_24h = model.predict(test_24h_scaled)
    predictions_original = scalers['electricity_load'].inverse_transform(predictions_24h)
    predictions_original = predictions_original.reshape(-1).tolist()
    actual_values = df_pse['electricity_load'].tail(24).values
    pse_predictions = df_pse['forecasted_load'].tail(24).values

    plt.figure(figsize=(15, 7))
    plt.plot(actual_values, label='Rzeczywiste zapotrzebowanie', color='green', linewidth=1.8)
    plt.plot(pse_predictions, label='Predykcja PSE', color='red', linewidth=0.8)
    plt.plot(predictions_original, label='Predykcja modelu', color='blue', linewidth=0.8)
    plt.xlabel('Godzina')
    plt.ylabel('Zapotrzebowanie [MW]')
    plt.legend()
    plt.grid(True)
    plt.show()

    print("MAPE predykcji opracowanego modelu:", mape(actual_values, predictions_original))
    print("MAPE predykcji PSE:", mape(actual_values, pse_predictions))
    print('RMSE predykcji opracowanego modelu:', np.sqrt(mean_squared_error(actual_values, predictions_original)))
    print('RMSE predykcji PSE: ', np.sqrt(mean_squared_error(actual_values, pse_predictions)))

# Wizualizacja funkcji strat w procesie uczenia dla poszczególnych modeli

In [ ]:
def plot_history(hist):
    plt.figure(figsize=(12, 6))
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.ylabel('Strata')
    plt.xlabel('Epoka')
    plt.legend(['Zestaw treningowy', 'Zestaw walidacyjny'], loc='upper right')
    plt.show()

---------------------------------------
# Utworzenie oraz uczenie sieci FFANN
---------------------------------------

# Model 1

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = Sequential()
model.add(Flatten(input_shape=(24, 7)))
model.add(Dense(168, activation='relu'))
model.add(Dense(126, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(24, activation='linear'))
model.compile(optimizer='adam', loss=rmse)

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc = ModelCheckpoint('model_1_ffann.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=64, validation_data=(X_val_seq, y_val_seq), callbacks=[es, mc], shuffle=False)

plot_history(history)

plots_and_metrics(model)

# Model 2

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(24, 7)))
model.add(Dense(168, activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Dense(72, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(24, activation='linear'))
model.compile(optimizer='adam', loss=rmse)

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc = ModelCheckpoint('model_2_ffann.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=64, validation_data=(X_val_seq, y_val_seq), callbacks=[es, mc], shuffle=False)

plot_history(history)

plots_and_metrics(model)

# Model 3

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(24, 7)))
model.add(Dense(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(24, activation='linear'))
model.compile(optimizer='adam', loss=rmse)

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc = ModelCheckpoint('model_3_ffann', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=64, validation_data=(X_val_seq, y_val_seq), callbacks=[es, mc], shuffle=False)

plot_history(history)

plots_and_metrics(model)

---------------------------------------
# Utworzenie oraz uczenie sieci FFANN wraz z optymalizacją PSO
---------------------------------------

# Model 4, 5, 6

In [ ]:
import random


NEURONS_RANGE = (25, 200)


def create_model(neurons_per_layer):
    model = Sequential()
    model.add(Flatten(input_shape=(24, 7)))
    for neurons in neurons_per_layer:
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(24, activation='linear'))
    model.compile(optimizer='adam', loss=rmse)
    model.summary()
    return model


def random_hyperparameters(n_layers):
    hyperparameters = []
    for _ in range(n_layers - 1):
        neurons = random.randint(*NEURONS_RANGE)
        hyperparameters.append(neurons)
    hyperparameters.append(random.randint(*NEURONS_RANGE))
    return hyperparameters


def random_velocity():
    v_neurons = random.uniform(-1, 1)
    return v_neurons


def update_velocity(particle, global_best_position, w=0.5, c1=1, c2=2):
    new_velocity = []
    for i, v_neurons in enumerate(particle.velocity):
        r1 = random.random()
        r2 = random.random()
        best_neurons = particle.best_position[i]
        current_neurons = particle.position[i]
        if global_best_position is not None and i < len(global_best_position):
            global_best_neurons = global_best_position[i]
        else:
            global_best_neurons = current_neurons
        cognitive_velocity_neurons = c1 * r1 * (best_neurons - current_neurons)
        social_velocity_neurons = c2 * r2 * (global_best_neurons - current_neurons)
        v_neurons_updated = w * v_neurons + cognitive_velocity_neurons + social_velocity_neurons
        new_velocity.append(v_neurons_updated)
    particle.velocity = new_velocity


def update_position(particle):
    new_position = []
    for i, v_neurons in enumerate(particle.velocity):
        current_neurons = particle.position[i]
        new_neurons = max(min(current_neurons + v_neurons, NEURONS_RANGE[1]), NEURONS_RANGE[0])
        new_position.append(new_neurons)
    particle.position = new_position


class Particle:
    def __init__(self, n_layers):
        self.position = random_hyperparameters(n_layers)
        self.velocity = [random_velocity() for _ in range(n_layers)]
        self.best_position = self.position
        self.best_error = float('inf')


def pso(n_particles, n_iterations, X_train, y_train, X_val, y_val, X_test, y_test):
    global_best_position = None
    global_best_error = float('inf')
    no_improve_count = 0
    particles = [Particle(random.randint(1, 5)) for _ in range(n_particles)]
    for iteration in range(n_iterations):
        improved = False
        for particle in particles:
            model = create_model(particle.position)
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7, restore_best_weights=True)
            history_best = model.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_val, y_val), callbacks=[es], shuffle=False)
            predictions = model.predict(X_test)
            error = rmse(y_test, predictions)
            if error < particle.best_error:
                particle.best_error = error
                particle.best_position = particle.position
            if error < global_best_error:
                global_best_error = error
                global_best_position = particle.best_position.copy()
                model.save('pso_best_model.h5')
                plt.figure(figsize=(12, 6))
                plt.plot(history_best.history['loss'])
                plt.plot(history_best.history['val_loss'])
                plt.ylabel('Strata')
                plt.xlabel('Epoka')
                plt.legend(['Zestaw treningowy', 'Zestaw walidacyjny'], loc='upper right')
                plt.savefig('pso_best_model_history.png')
                plt.close()
                improved = True
                no_improve_count = 0

        if not improved:
            no_improve_count += 1

        if no_improve_count >= 5:
            print(f"Brak poprawy globalnej straty przez {iteration+1}. Zatrzymywanie działania algorytmu PSO...")
            return global_best_position

        for particle in particles:
            update_velocity(particle, global_best_position)
            update_position(particle)
        print(f"Iteration {iteration+1}, Best Error: {global_best_error}")
    return global_best_position


hyperparameters = pso(n_particles=15, n_iterations=40, X_train=X_train_seq, y_train=y_train_seq, X_val=X_val_seq, y_val=y_val_seq, X_test=X_test_seq,y_test=y_test_seq)
print(hyperparameters)

model = tf.keras.models.load_model('pso_best_model.h5', custom_objects={'rmse': rmse})

plot_history(history)

plots_and_metrics(model)

---------------------------------------
# Utworzenie oraz uczenie sieci LSTM
---------------------------------------

# Model 7

In [ ]:
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(LSTM(32, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(24))
model.compile(optimizer='adam', loss=rmse)

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7, restore_best_weights=True)
mc = ModelCheckpoint('model_7_lstm.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train_seq, y_train_seq, epochs=1, batch_size=16, validation_data=(X_val_seq, y_val_seq), callbacks=[es, mc], shuffle=False)

plot_history(history)

plots_and_metrics(model)

# Model 8

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(24))
model.compile(optimizer='adam', loss=rmse)

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7, restore_best_weights=True)
mc = ModelCheckpoint('model_8_lstm.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=16, validation_data=(X_val_seq, y_val_seq), callbacks=[es, mc], shuffle=False)

plot_history(history)

plots_and_metrics(model)

# Model 9

In [ ]:
model = Sequential()
model.add(LSTM(168, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(24))
model.compile(optimizer='adam', loss=rmse)

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7, restore_best_weights=True)
mc = ModelCheckpoint('model_9_lstm.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=16, validation_data=(X_val_seq, y_val_seq), callbacks=[es, mc], shuffle=False)
plot_history(history)

plots_and_metrics(model)